<a href="https://colab.research.google.com/github/shruti629/Deep-Health-Analysis-and-Disease-Prediction/blob/main/Food_Based_upon_Disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler
import pickle
import warnings
warnings.filterwarnings('ignore')

In [ ]:
fd=pd.read_csv('/content/drive/MyDrive/food prediction.csv')
fd.head()

,Diseases,Recommended Foods,Foods to Avoid,Exercise Recommendations
0,Fever,"Hydrating fluids (water, herbal teas, broths),...","Heavy, greasy foods, dairy (if it causes mucus...","Gentle exercises (stretching, light walking), ..."
1,Diarrhea,"BRAT diet (bananas, rice, applesauce, toast), ...","Dairy, fatty foods, high-fiber foods, caffeine...","Gentle exercises (light walking, stretching), ..."
2,Constipation,"High-fiber foods (whole grains, fruits, vegeta...","Low-fiber foods, processed foods, excessive da...","Aerobic exercises (walking, swimming), stretch..."
3,Headache,"Hydrating fluids (water, herbal teas), magnesi...","Alcohol, caffeine (in excess), processed foods...","Gentle exercises (walking, yoga), relaxation e..."
4,Common Cold,"Hydrating fluids (water, herbal teas, broths),...","Dairy (if it causes mucus), sugary foods, caff...","Gentle exercises (walking, stretching), rest a..."


In [ ]:
# Encode categorical variables using OneHotEncoder for disease
onehot_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
disease_encoded = onehot_encoder.fit_transform(fd[['Diseases']])


In [ ]:
# Append encoded columns to the dataframe
disease_encoded_df = pd.DataFrame(disease_encoded, columns=onehot_encoder.get_feature_names_out(['Diseases']))
fd = pd.concat([fd, disease_encoded_df], axis=1)


In [ ]:
fd.head()

,Diseases,Recommended Foods,Foods to Avoid,Exercise Recommendations,Diseases_Alzheimer's Disease,Diseases_Arthritis,Diseases_Asthma,Diseases_Avian Influenza,Diseases_Bladder Cancer,Diseases_Breast Cancer,...,Diseases_Prostate Cancer,Diseases_Psoriasis,Diseases_RSV (Respiratory Syncytial Virus),Diseases_Skin Cancer,Diseases_Sore Throat,Diseases_Stomach Cancer,Diseases_Stroke,Diseases_Typhoid Fever,Diseases_Yellow Fever,Diseases_Zika Virus
0,Fever,"Hydrating fluids (water, herbal teas, broths),...","Heavy, greasy foods, dairy (if it causes mucus...","Gentle exercises (stretching, light walking), ...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Diarrhea,"BRAT diet (bananas, rice, applesauce, toast), ...","Dairy, fatty foods, high-fiber foods, caffeine...","Gentle exercises (light walking, stretching), ...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Constipation,"High-fiber foods (whole grains, fruits, vegeta...","Low-fiber foods, processed foods, excessive da...","Aerobic exercises (walking, swimming), stretch...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Headache,"Hydrating fluids (water, herbal teas), magnesi...","Alcohol, caffeine (in excess), processed foods...","Gentle exercises (walking, yoga), relaxation e...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Common Cold,"Hydrating fluids (water, herbal teas, broths),...","Dairy (if it causes mucus), sugary foods, caff...","Gentle exercises (walking, stretching), rest a...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Separate the data for each target variable
X = fd.drop(columns=['Recommended Foods', 'Foods to Avoid', 'Exercise Recommendations'])
y_recommended_food = fd['Recommended Foods']
y_foods_to_avoid = fd['Foods to Avoid']
y_exercise_recommendations = fd['Exercise Recommendations']

In [ ]:
# Encode target variables using LabelEncoder
le_recommended_food = LabelEncoder()
le_foods_to_avoid = LabelEncoder()
le_exercise_recommendations = LabelEncoder()

y_recommended_food = le_recommended_food.fit_transform(y_recommended_food)
y_foods_to_avoid = le_foods_to_avoid.fit_transform(y_foods_to_avoid)
y_exercise_recommendations = le_exercise_recommendations.fit_transform(y_exercise_recommendations)


In [ ]:
# Address class imbalance using RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_resampled_rf, y_resampled_rf = ros.fit_resample(X, y_recommended_food)
X_resampled_fta, y_resampled_fta = ros.fit_resample(X, y_foods_to_avoid)
X_resampled_er, y_resampled_er = ros.fit_resample(X, y_exercise_recommendations)


In [ ]:
# Split the data into training and testing sets
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_resampled_rf, y_resampled_rf, test_size=0.2, random_state=42)
X_train_fta, X_test_fta, y_train_fta, y_test_fta = train_test_split(X_resampled_fta, y_resampled_fta, test_size=0.2, random_state=42)
X_train_er, X_test_er, y_train_er, y_test_er = train_test_split(X_resampled_er, y_resampled_er, test_size=0.2, random_state=42)


In [ ]:
# Import necessary libraries
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Assuming 'Diseases' is the categorical column containing strings like 'Diarrhea'
categorical_features = ['Diseases']  # Replace with the actual names of your categorical columns

# Create a transformer to handle categorical features
# handle_unknown='ignore' tells the encoder to ignore unknown categories during transform
transformer = ColumnTransformer(
    transformers=[('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_features)],
    remainder='passthrough'  # Keep numerical features as they are
)


In [ ]:
# Apply the transformation to your training and testing data
X_train_rf_encoded = transformer.fit_transform(X_train_rf)
X_test_rf_encoded = transformer.transform(X_test_rf)

X_train_fta_encoded = transformer.fit_transform(X_train_fta)
X_test_fta_encoded = transformer.transform(X_test_fta)

X_train_er_encoded = transformer.fit_transform(X_train_er)
X_test_er_encoded = transformer.transform(X_test_er)

# Now train your models using the encoded data
model_rf.fit(X_train_rf_encoded, y_train_rf)
model_fta.fit(X_train_fta_encoded, y_train_fta)
model_er.fit(X_train_er_encoded, y_train_er)


RandomForestClassifier()

In [ ]:
# Function to make predictions
def predict_recommendations(disease_name):
    try:
        # Ensure the disease name is properly formatted
        disease_name = disease_name.strip()
        disease_encoded = onehot_encoder.transform([[disease_name]])

        # Align the encoded features with the training data
        disease_encoded_df = pd.DataFrame(disease_encoded, columns=onehot_encoder.get_feature_names_out(['Diseases']))

        # Ensure the feature names match exactly
        X_train_columns = X.columns
        disease_encoded_df = disease_encoded_df.reindex(columns=X_train_columns, fill_value=0)

        recommended_food_encoded = model_rf.predict(disease_encoded_df)
        foods_to_avoid_encoded = model_fta.predict(disease_encoded_df)
        exercise_recommendations_encoded = model_er.predict(disease_encoded_df)

        recommended_food = le_recommended_food.inverse_transform(recommended_food_encoded)
        foods_to_avoid = le_foods_to_avoid.inverse_transform(foods_to_avoid_encoded)
        exercise_recommendations = le_exercise_recommendations.inverse_transform(exercise_recommendations_encoded)

        return recommended_food[0], foods_to_avoid[0], exercise_recommendations[0]
    except ValueError as e:
        return str(e), str(e), str(e)

In [ ]:
# Example prediction
example_disease = 'Diabetes'  # Replace with a disease name from your dataset
predicted_recommended_food, predicted_foods_to_avoid, predicted_exercise_recommendations = predict_recommendations(example_disease)
print(f'Predicted Recommended Food for {example_disease}: {predicted_recommended_food}')
print(f'Predicted Foods to Avoid for {example_disease}: {predicted_foods_to_avoid}')
print(f'Predicted Exercise Recommendations for {example_disease}: {predicted_exercise_recommendations}')

Predicted Recommended Food for Diabetes: X has 62 features, but RandomForestClassifier is expecting 115 features as input.
Predicted Foods to Avoid for Diabetes: X has 62 features, but RandomForestClassifier is expecting 115 features as input.
Predicted Exercise Recommendations for Diabetes: X has 62 features, but RandomForestClassifier is expecting 115 features as input.
